In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
from openai import OpenAI

openai_client = OpenAI()

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
rag('how do I run kafka?')

"To run Kafka, the specific instructions depend on the type of application you are using (Java or Python). For a Java-based Kafka application, you can run a producer or consumer by using the command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor a Python Kafka application, you should first set up a virtual environment and install the required packages. Here's how you can do it:\n\n1. Create a virtual environment (run only once):\n   ```\n   python -m venv env\n   ```\n\n2. Activate the virtual environment:\n   - On MacOS or Linux:\n     ```\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```\n     env\\Scripts\\activate\n     ```\n\n3. Install the required packages:\n   ```\n   pip install -r ../requirements.txt\n   ```\n\n4. When you're done, deactivate the virtual environment using:\n   ```\n   deactivate\n   ```"

In [11]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register right away, you are eligible to submit the homework. However, keep in mind that there are deadlines for submitting the final projects, so it's important not to leave everything until the last minute."

# RAG with Vector Search

In [14]:
from qdrant_client import QdrantClient, models

In [13]:
qd_client = QdrantClient("http://localhost:6333")

In [17]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [15]:
collection_name = "zoomcamp-faq"

In [16]:
qd_client.delete_collection(collection_name=collection_name)

True

In [17]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [18]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [20]:
model_handle

'jinaai/jina-embeddings-v2-small-en'

In [21]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
question = 'I just discovered the course. Can I still join it?'

In [36]:
def vector_search(question):
    print("vector search is used")

    course="data-engineering-zoomcamp"

    query_points=qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    results=[]
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [39]:
def rag(question):
    search_results=vector_search(question)
    prompt=build_prompt(question, search_results)
    answer=llm(prompt)
    return answer

In [40]:
rag("what is etl pipeline?")

vector search is used


'The provided CONTEXT does not contain any information specifically defining or explaining what an ETL pipeline is. Therefore, I cannot provide an answer to the QUESTION based on the available CONTEXT.\n\nFor general knowledge, an ETL pipeline typically refers to a data processing pipeline that involves Extracting data from various sources, Transforming it into a suitable format or structure for the intended purpose, and Loading it into a final destination system or database.'

# Homework 2 with FastEmbedding

In [15]:
!pip install fastembed


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
from fastembed import TextEmbedding
import numpy as np

### No. 1 Embedding the query

In [19]:
embedding_model = TextEmbedding(model_name=model_handle, size=EMBEDDING_DIMENSIONALITY)
embedding_model

Fetching 5 files: 100%|████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.73s/it]


In [45]:
embedded_vector = list(embedding_model.embed('I just discovered the course. Can I join now?'))
print(min(embedded_vector[0]))

-0.11726373885183883


The min value of the embedded vector is -0.11.

### No. 2 Cosine similarity with another vector

In [46]:
import numpy as np
np.linalg.norm(embedded_vector)

np.float64(1.0)

In [47]:
doc = 'Can I still join the course after the start date?'

In [49]:
embedded_doc = list(embedding_model.embed(doc))
np.array(embedded_vector).dot(embedded_doc[0])

array([0.90085289])

 the cosine similarity between the vector for the query and the vector for the documen is 0.9

### No.3 Ranking by cosine

In [50]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [63]:
embedded_docs = []
for i,doc in enumerate(documents):
    embedded_docs.append(list(embedding_model.embed(doc['text']))[0])

np.array(embedded_docs).dot(np.array(embedded_vector).reshape(512,))

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

The document index with the highest similarity is  1.

### No. 4 Ranking by cosine, version two

In [66]:
full_text = doc['question'] + ' ' + doc['text']
full_text

'How can we contribute to the course? Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'

In [68]:
np.array(embedded_docs).dot(list(embedding_model.embed(full_text))[0])

array([0.73050042, 0.7998077 , 0.79154204, 0.76257202, 0.94839479])

The highest scoring document is  4.

### No.5 Selecting the embedding model

The smallest dimensionality for models in fastembed is 384.

In [70]:
smallest_model = 'BAAI/bge-small-en-v1.5'

### No.6 Indexing with qdrant

In [ ]:
import requests 
from qdrant_client import QdrantClient, models

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


qd_client = QdrantClient("http://localhost:6333")
qd_client.delete_collection(collection_name="homework2_llmzoomcamp")
qd_client.create_collection(
        collection_name="homework2_llmzoomcamp",
        vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)
qd_client.create_payload_index(
    collection_name="homework2_llmzoomcamp",
    field_name="course",
    field_schema="keyword"
)
print("Done creating collection!")
      
points = []
for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=smallest_model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name="homework2_llmzoomcamp",
    points=points
)
print("Done upserting!")



In [90]:
def vector_search(question):
    query_points=qd_client.query_points(
        collection_name="homework2_llmzoomcamp",
        query=models.Document(
            text=question,
            model=smallest_model
        ),
        limit=5,
        with_payload=True
    )

    results=[]
    for point in query_points.points:
        results.append(point.payload)
    
    return results, query_points.points

results, points = vector_search('I just discovered the course. Can I join now?')
print("Done Searching!")

Done Searching!


In [91]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Here’s how you join a in Slack: https://slack.com/help/articles/205239967-Join-a-channel\nClick “All channels” at the top of your left sidebar. If you don't see this option, click “More” to find it.\nBrowse the list of public channels in your workspace, or use the search bar to search by channel name or description.\nSelect a channel from the list to view it.\nClick Join Channel.\nDo we need to provide the GitHub link to only our code co

In [92]:
points

[ScoredPoint(id=14, version=2, score=0.7397805, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13, version=2, score=0.72352135, payload={'text': "Here’s how you join a in Slack: https://slack.com/help/articles/205239967-Join-a-channel\nClick “All channels” at the top of your left sidebar. If you don't see this option, click “More” to find it.\nBrowse the list of public channels in your workspace, or use the search bar to search by